In [1]:
import numpy as np
import pandas as pd
import IPython.display as ipd
import matplotlib.pyplot as plt

import scipy 
import scipy.fft

import librosa
import librosa.display
import librosa.filters
import librosa.feature

import os
import sklearn

import pickle

%matplotlib inline

## Загружаю расчитанные фичи и словарь меток.

In [2]:
def pkl_load(file):
    return pickle.load(open(file,'rb'))

In [3]:
from sklearn.model_selection import train_test_split

data = pd.DataFrame(pkl_load('../audioset/Processed/features.pkl'))
actual_dict = pkl_load('../audioset/Processed/actual_dict.pkl')

## Разбиваю на тренировачную тестовую выборки.

In [4]:
x_train, x_test, y_train, y_test = train_test_split(data['features'].values, data['label'].values, test_size=0.25)
np.asarray(x_test).shape

(83137,)

## Код для обучения Random forest

* Создание классификатора с задаными параметрами (кол-во деревьев, глубина деревьев).
* Перераспределение данных для использования на эпохах обучения.
* Обучение (заполнение модели, определение промежуточного результата).

In [19]:
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier

def make_training_model(x, y, epoch_cnt, n_estimators, forest_depth):
    classifier = RandomForestClassifier(n_estimators=n_estimators, max_depth=forest_depth, n_jobs=-1)
    
    Selector = KFold(n_splits=epoch_cnt, random_state=None, shuffle=False)
    
    print('\n\n', classifier)
    
    Number = 0
    for train_index, test_index in Selector.split(X=x, y=y):   
        Number += 1

        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        classifier.fit(x_train.tolist(), y_train.tolist())
        print('\nBlock', Number, ' score:', metrics.f1_score(y_test.tolist(), classifier.predict(x_test.tolist()), average='micro'))
        
    return classifier

Первый эксперимент. Попробовал взять кол-во деревьев равное кол-ву меток. И небольшую глубину деревьев.

In [20]:
classifier = make_training_model(x_train, y_train, epoch_cnt=8, n_estimators=len(actual_dict), forest_depth=4)
first_score = metrics.f1_score(y_test.tolist(), classifier.predict(x_test.tolist()), average='micro')
print('\nResult score:', first_score)



 RandomForestClassifier(max_depth=4, n_estimators=61, n_jobs=-1)

Block 1  score: 0.2694701052091352

Block 2  score: 0.2659096741082884

Block 3  score: 0.2687002822684116

Block 4  score: 0.2653643828586092

Block 5  score: 0.2679946112394149

Block 6  score: 0.2583718244803695

Block 7  score: 0.2691172696946369

Block 8  score: 0.2627662304336669

Result score: 0.264515197806031


Увеличиваю глубину деревьев -> результаты улучшаются

In [21]:
classifier = make_training_model(x_train, y_train, epoch_cnt=8, n_estimators=len(actual_dict), forest_depth=6)
print('\nResult score:', metrics.f1_score(y_test.tolist(), classifier.predict(x_test.tolist()), average='micro'))



 RandomForestClassifier(max_depth=6, n_estimators=61, n_jobs=-1)

Block 1  score: 0.27133051064921737

Block 2  score: 0.26748139594559917

Block 3  score: 0.27062483962022066

Block 4  score: 0.26728894021041827

Block 5  score: 0.2703361560174493

Block 6  score: 0.26023222992045164

Block 7  score: 0.2712663587374904

Block 8  score: 0.26398511675647934

Result score: 0.26623525024958805


Немного увеличиваю глубину и значительно кол-во деревьев. -> результаты меняются не сильно.

In [22]:
classifier = make_training_model(x_train, y_train, epoch_cnt=8, n_estimators=len(actual_dict)*4, forest_depth=10)
print('\nResult score:', metrics.f1_score(y_test.tolist(), classifier.predict(x_test.tolist()), average='micro'))



 RandomForestClassifier(max_depth=10, n_estimators=244, n_jobs=-1)

Block 1  score: 0.29606107261996406

Block 2  score: 0.29548370541442137

Block 3  score: 0.2953874775468309

Block 4  score: 0.2943289710033359

Block 5  score: 0.29673466769309725

Block 6  score: 0.28640620990505516

Block 7  score: 0.2997818835001283

Block 8  score: 0.2899345650500385

Result score: 0.29332306915091955


Так как увеличение кол-ва деревьев ни к чему не привело. Пробую увеличить глубину -> результаты заметно лучше.

In [23]:
classifier = make_training_model(x_train, y_train, epoch_cnt=8, n_estimators=len(actual_dict), forest_depth=16)
print('\nResult score:', metrics.f1_score(y_test.tolist(), classifier.predict(x_test.tolist()), average='micro'))



 RandomForestClassifier(max_depth=16, n_estimators=61, n_jobs=-1)

Block 1  score: 0.3722735437516038

Block 2  score: 0.3711188093405184

Block 3  score: 0.375930202720041

Block 4  score: 0.37025275853220424

Block 5  score: 0.3730754426481909

Block 6  score: 0.36476777007954836

Block 7  score: 0.3760905824993585

Block 8  score: 0.36832820118039516

Result score: 0.3697631620096949


Значительно уменьшаю кол-во деревьев и сильно увеличевую глубину -> результаты немного лучше

In [24]:
classifier = make_training_model(x_train, y_train, epoch_cnt=8, n_estimators=10, forest_depth=24)
print('\nResult score:', metrics.f1_score(y_test.tolist(), classifier.predict(x_test.tolist()), average='micro'))



 RandomForestClassifier(max_depth=24, n_estimators=10, n_jobs=-1)

Block 1  score: 0.39382858609186555

Block 2  score: 0.38985116756479343

Block 3  score: 0.393988965871183

Block 4  score: 0.39033230690274573

Block 5  score: 0.39540030792917635

Block 6  score: 0.3905889145496536

Block 7  score: 0.3933795227097767

Block 8  score: 0.39193610469591994

Result score: 0.3902233662508871


Пытаюсь еще увеличить глубину. Результыты не меняются. Значит дальше будем менять другие два параметра.

In [25]:
classifier = make_training_model(x_train, y_train, epoch_cnt=8, n_estimators=10, forest_depth=64)
print('\nResult score:', metrics.f1_score(y_test.tolist(), classifier.predict(x_test.tolist()), average='micro'))



 RandomForestClassifier(max_depth=64, n_estimators=10, n_jobs=-1)

Block 1  score: 0.3900436232999743

Block 2  score: 0.3878624583012574

Block 3  score: 0.3945663330767257

Block 4  score: 0.38856812933025403

Block 5  score: 0.3927059276366436

Block 6  score: 0.38564921734667695

Block 7  score: 0.39206440851937385

Block 8  score: 0.3885360533743906

Result score: 0.3889363340029108


Снова пробую увеличить кол-во деревьев и кол-во эпох -> результаты лучше еще на первой эпохе. Значит кол-во эпох не сильно повлияло на результат.

In [26]:
classifier = make_training_model(x_train, y_train, epoch_cnt=20, n_estimators=100, forest_depth=32)
print('\nResult score:', metrics.f1_score(y_test.tolist(), classifier.predict(x_test.tolist()), average='micro'))



 RandomForestClassifier(max_depth=32, n_jobs=-1)

Block 1  score: 0.4409429877315372

Block 2  score: 0.43990056932082433

Block 3  score: 0.44407024296367575

Block 4  score: 0.43244326838264774

Block 5  score: 0.43974019725763774

Block 6  score: 0.44150428995269025

Block 7  score: 0.4400609413840109

Block 8  score: 0.44238633630021645

Block 9  score: 0.43207698476343226

Block 10  score: 0.441299117882919

Block 11  score: 0.44394546912590216

Block 12  score: 0.42782678428227744

Block 13  score: 0.44065757818765033

Block 14  score: 0.4327185244587009

Block 15  score: 0.4312750601443464

Block 16  score: 0.4404971932638332

Block 17  score: 0.4366479550922213

Block 18  score: 0.4352044907778669

Block 19  score: 0.43696872493985567

Block 20  score: 0.43448275862068964

Result score: 0.4358348268520635


Итоговый эксперимент. Пробую еще увеличить кол-во деревьев -> изменений нет -> Значит с данными параметрами добились максимума.

In [ ]:
classifier = make_training_model(x_train, y_train, epoch_cnt=8, n_estimators=150, forest_depth=32)
last_score = metrics.f1_score(y_test.tolist(), classifier.predict(x_test.tolist()), average='micro')
print('\nResult score:', last_score)



 RandomForestClassifier(max_depth=32, n_estimators=150, n_jobs=-1)

Block 1  score: 0.44191044393122914

Block 2  score: 0.4388632281241981

Block 3  score: 0.4409802412111881

Block 4  score: 0.4364575314344367

Block 5  score: 0.436938670772389

Block 6  score: 0.4334103156274057

Block 7  score: 0.4404028740056454

Block 8  score: 0.43642545547857325


In [ ]:
print(first_score, '->', last_score)